# Pipeline

- This notebook is used to test various aspects of the twitter pipeline 

## Imports

In [1]:
path = '../'
import csv , dateutil.parser , time
from datetime import date , timedelta 
import os
# classifier
import pandas as pd
import pickle
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import classification_report 
from sklearn.model_selection import cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
# web scraping
import requests
from bs4 import BeautifulSoup
#!pip3 install trafilatura
import trafilatura
from transformers import pipeline
#cleaning 
import emoji
import re
#functions
from PYTHON_FILES.LogReg_Searches import LogRegSearches

## Variables

In [20]:
# descriptions training set -> v2 = musow+mji descriptions vs summarized scrapes from twitter searches  
archive_desc_training_v2 = pd.read_pickle(path+'LOGREG_RELEVANCE/TRAINING_SETS/archive_desc_training_v2.pkl')

# twitter training set -> v1 = tweets from bigrams vs tweets for digital humanities and music company 
twitter_training_set_v1 = pd.read_pickle(path+'LOGREG_RELEVANCE/TRAINING_SETS/twitter_training_v1.pkl')

## Functions

In [3]:
def lr_training(t_input, t_feature, target, cv_int, score_type, filename, path):
    """ Create a text classifier based on Logistic regression and TF-IDF. Use cross validation 
    
    Parameters
    ----------
    t_input: 
        dataframe of the training set
    t_feature: 
        df column, text of tweet or description of the resource
    target: 
        df column, [0,1] values
    cv_int: int
        the number of cross validation folding
    score_type: str
        precision or recall
    filename: str
        model file name
    path: str
        parent folder
    """
    # TODO eda to define max_features=1000
      
    tfidf_transformer = TfidfVectorizer(ngram_range=(1, 2), lowercase=True, max_features=1000) 
    x_train = tfidf_transformer.fit_transform(t_input[t_feature])
    y_train = t_input[target].values
    model = LogisticRegressionCV(solver='liblinear', random_state=44, cv=cv_int, scoring=score_type)
    
    # export
    model.fit(x_train, y_train)
    export_model = f'LOGREG_RELEVANCE/MODELS/{filename}_model.pkl'
    export_vectorizer = f'LOGREG_RELEVANCE/MODELS/{filename}_vectorizer.pkl'
    pickle.dump(model, open(path+export_model, 'wb'))
    pickle.dump(tfidf_transformer, open(path+export_vectorizer, 'wb'))
    
    # report
    y_pred = cross_val_predict(model, x_train, y_train, cv=cv_int)
    report = classification_report(y_train, y_pred)
    print('report:', report, sep='\n')
    return model
    
def lr_predict(path, filename, p_input, p_feature):
    """ Classify text using a pickled model based on Logistic regression and TF-IDF.
    
    Parameters
    ----------
    p_input: 
        dataframe of the prediction set
    p_feature: 
        df column, text of tweet or description of the resource
    filename: str
        model file name
    path: str
        parent folder
    """
    export_model = f'{path}LOGREG_RELEVANCE/MODELS/{filename}_model.pkl'
    export_vectorizer = f'{path}LOGREG_RELEVANCE/MODELS/{filename}_vectorizer.pkl'
    model = pickle.load(open(export_model, 'rb'))
    tfidf_transformer = pickle.load(open(export_vectorizer, 'rb'))
  
    #result = loaded_model.score(X_test, Y_test)
    #x_new_count = count_vect.transform(p_input[p_feature])
    x_predict = tfidf_transformer.transform(p_input[p_feature])
    y_predict = model.predict(x_predict)
    scores = model.decision_function(x_predict)
    probability = model.predict_proba(x_predict)
    
    #results = [r for r in y_predict]
    result = p_input.copy()
    result['Prediction'] = y_predict
    result['Score'] = scores
    result['Probability'] = probability[:,1]
    result['Input Length'] = result[p_feature].str.len()
    return result

def create_url(keyword, start_date, end_date, max_results):
        search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from
        #change params based on the endpoint you are using
        query_params = {'query': keyword,
                        'start_time': start_date,
                        'end_time': end_date,
                        'max_results': max_results,
                        'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                        'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source,entities',
                        'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                        'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                        'next_token': {}}
        return (search_url, query_params)
    
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def give_emoji_free_text(text):
    return emoji.replace_emoji(text, replace='')

def append_to_csv(json_response, fileName):
    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)
    
    #setup usernames via includes
    username = {user['id']: user['username'] for user in json_response['includes']['users']}
    
    #Loop through each tweet
    for tweet in json_response['data']:

        # 1. Username
        author_id = tweet['author_id']
        user = username[author_id]

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Language
        lang = tweet['lang']

        # 4. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        #5. URLs w/ a catch for tweets w/ two links TODO: how to catch more than two links? 
        if ('entities' in tweet) and ('urls' in tweet['entities']):
            for url in tweet['entities']['urls']:
                url = [url['expanded_url'] for url in tweet['entities']['urls'] if 'twitter.com' not in url['expanded_url']]
                url = ', '.join(url)
        else:
            url = ""
        
        #6. Tweet text
        text = give_emoji_free_text(tweet['text']) 
        
        # Assemble all data in a list
        res = [user, created_at, lang, like_count, quote_count, reply_count, retweet_count, text, url]

        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1    
    
    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

def twitter_search(token, keyword, start, end, mresults, mcount, file_name):
    
    # TODO filter tweets in english only OR tweak TF-IDF stopwords (lang detection)
    bearer_token = token
    headers = {"Authorization": "Bearer {}".format(bearer_token)} 
    start_list = start
    end_list =  end
    max_results = mresults
    total_tweets = 0

    # Create file
    csvFile = open(f'{path}TWITTER_SEARCHES/RAW_SEARCHES/{file_name}.csv', "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)
    csvWriter.writerow(['user', 'created_at', 'lang', 'like_count', 'quote_count', 'reply_count','retweet_count','tweet', 'URL'])
    csvFile.close()

    for i in range(0,len(start_list)):
        # Inputs
        count = 0 # Counting tweets per time period
        max_count = mcount # Max tweets per time period
        flag = True
        next_token = None
        
        while flag:
            # Check if max_count reached
            if count >= max_count:
                break
            print("-------------------")
            print("Token: ", next_token)
            url = create_url(keyword, start_list[i],end_list[i], max_results)
            json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
            result_count = json_response['meta']['result_count']

            if 'next_token' in json_response['meta']:
                # Save the token to use for next call
                next_token = json_response['meta']['next_token']
                print("Next Token: ", next_token)
                if result_count is not None and result_count > 0 and next_token is not None:
                    print("Start Date: ", start_list[i])
                    append_to_csv(json_response, f'{path}TWITTER_SEARCHES/RAW_SEARCHES/{file_name}.csv')
                    count += result_count
                    total_tweets += result_count
                    print(f"Total # of Tweets added for '{keyword}':", total_tweets)
                    print("-------------------")
                    time.sleep(5)                
            # If no next token exists
            else:
                if result_count is not None and result_count > 0:
                    print("-------------------")
                    print("Start Date: ", start_list[i])
                    append_to_csv(json_response, f'{path}TWITTER_SEARCHES/RAW_SEARCHES/{file_name}.csv')
                    count += result_count
                    total_tweets += result_count
                    print(f"Total # of Tweets added for '{keyword}':", total_tweets)
                    print("-------------------")
                    time.sleep(5)

                #Since this is the final request, turn flag to false to move to the next time period.
                flag = False
                next_token = None
            time.sleep(5)
    print("Total number of results:", total_tweets)
    
    df = pd.read_csv(f'{path}TWITTER_SEARCHES/RAW_SEARCHES/{file_name}.csv', keep_default_na=False, dtype={"user": "string", "lang": "string", "tweet": "string", "URL": "string"})
    
    # clean the tweet from meentions, hashtags, emojis
    df['tweet'].replace( { r"@[A-Za-z0-9_]+": '' }, inplace= True, regex = True)
    df['tweet'].replace( { r"#": '' }, inplace= True, regex = True)
    
    # remove tweets that are not in english, have empty URLs, or have duplicate URLs
    df = df[df['lang'].isin(['en'])]
    df = df[df.URL != '']
    df = df.drop_duplicates(['URL'], keep='last')

    #add a column for the search keyword
    df['Search KW'] = keyword

    #pickle df for reuse
    df.to_pickle(f'{path}TWITTER_SEARCHES/RAW_SEARCHES/{file_name}.pkl')

def scrape_links(link_list, pred_df, filename):
    """ Scrape links from classified tweets, save scrapes, combine them w/ tweets and return a DF for description classification.
    
    Parameters
    ----------
    p_input: 
        dataframe of the prediction set
    p_feature: 
        df column, text of tweet or description of the resource
    filename: str
        model file name
    path: str
        parent folder
    score: int
        which prediction score to filter the results by 1/0
    discard: variable
        a list of terms to check against to remove tweets
    filter: str 
        a string against which to further filter predictions 
    """
    links = pd.DataFrame(columns=['Title', 'Description', 'URL'])
    summarizer = pipeline("summarization", model='sshleifer/distilbart-cnn-12-6')
    
    for link in link_list:
        URL = link
        page = None
        ARTICLE = ''
        try:
            x = requests.head(URL, timeout=15)
            content_type = x.headers["Content-Type"] if "Content-Type" in x.headers else "None"
            if ("text/html" in content_type.lower()):
                page = requests.get(URL, timeout=15)
        except Exception:
            pass
        
        if page:
            soup = BeautifulSoup(page.content, "html.parser")
            title = ' '.join([t.text for t in soup.find('head').find_all('title')]).strip() \
                if soup and soup.find('head') and soup.find('body') is not None \
                else URL
            
            try:
                downloaded = trafilatura.fetch_url(URL)
                ARTICLE = trafilatura.extract(downloaded, include_comments=False, include_tables=True, target_language='en', deduplicate=True)
            except Exception:
                results = soup.find_all(['h1', 'p'])
                text = [result.text for result in results]
                ARTICLE = ' '.join(text)
            
            if ARTICLE is not None and len(ARTICLE) > 200:
                # text summarisation
                max_chunk = 500
                #removing special characters and replacing with end of sentence
                ARTICLE = ARTICLE.replace('.', '.<eos>')
                ARTICLE = ARTICLE.replace('?', '?<eos>')
                ARTICLE = ARTICLE.replace('!', '!<eos>')
                sentences = ARTICLE.split('<eos>')
                current_chunk = 0 
                chunks = []

                # split text to process
                for sentence in sentences:
                    if len(chunks) == current_chunk + 1: 
                        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
                            chunks[current_chunk].extend(sentence.split(' '))
                        else:
                            current_chunk += 1
                            chunks.append(sentence.split(' '))
                    else:
                        chunks.append(sentence.split(' '))

                for chunk_id in range(len(chunks)):
                    chunks[chunk_id] = ' '.join(chunks[chunk_id])
                try:
                    res = summarizer(chunks, min_length = int(0.2 * len(text)), max_length = int(0.5 * len(text)), do_sample=False)
                    # summary
                    text = ' '.join([summ['summary_text'] for summ in res])
                except Exception:
                    text = ARTICLE
                    continue
            else:
                text = ARTICLE
            print(URL)
            new_row = {'Title': title, 'Description': text, 'URL': URL.strip()}
            new_df = pd.DataFrame(data=new_row, index=[0])
            links = pd.concat([links, new_df], ignore_index=True)
    discard = ['None', '! D O C T Y P E h t m l >', '! d o c t y p e h t m l >', '! D O C T Y P E H T M L >']
    links = links.fillna('None')
    links = links[~links.Description.str.contains('|'.join(discard))]
    twitter_scrapes_preds = pd.merge(pred_df, links, on='URL')
    twitter_scrapes_preds.to_pickle(f'{path}LOGREG_RELEVANCE/SCRAPES/{filename}.pkl')
    print(len(twitter_scrapes_preds))
    return twitter_scrapes_preds

def twitter_predictions(path, filename, p_input, p_feature, score, discard, filter):
    """ Predict relevant tweets using a pickled model based on Logistic regression and TF-IDF.
    
    Parameters
    ----------
    p_input: 
        dataframe of the prediction set
    p_feature: 
        df column, text of tweet or description of the resource
    filename: str
        model file name
    path: str
        parent folder
    score: int
        which prediction score to filter the results by 1/0
    discard: variable
        a list of terms to check against to remove tweets
    filter: str 
        a string against which to further filter predictions 
    """
    preds = lr_predict(path, filename, p_input, p_feature)
    preds = preds.drop_duplicates(['tweet'], keep='last')
    preds = preds.loc[preds['Prediction'] == score]
    preds = preds[~preds.URL.str.contains('|'.join(discard))]
    preds = preds.sort_values(by='Score', ascending=False).reset_index(drop=True)
    preds = preds[['tweet', 'Prediction', 'Score', 'Probability', 'Input Length', 'URL', 'Search KW']]
    if filter != '':
        preds = preds[preds['tweet'].str.contains(filter)]
        preds = preds.reset_index(drop=True)
    twitter_link_list = [link for link in preds['URL']]
    print('Total tweets classified:', len(preds))
    return preds, twitter_link_list

def resource_predictions(path, filename, p_input, p_feature, score, discard, savefile):
    """ Predict relevant URL descriptions using a pickled model based on Logistic regression and TF-IDF.
    
    Parameters
    ----------
    p_input: 
        dataframe of the prediction set
    p_feature: 
        df column, text of tweet or description of the resource
    filename: str
        model file name
    path: str
        parent folder
    score: int
        which prediction score to filter the results by 1/0
    discard: variable
        a list of terms to check against to remove tweets
    savefile: str
        name for the final csv to be saved under 
    """
    if len(filename) == 0:
        return 'Sorry no URLs to classify!'
    preds = lr_predict(path, filename, p_input, p_feature)
    preds = preds.drop_duplicates(['Description'], keep='last')
    preds = preds.loc[preds['Description'] != '']
    preds = preds.loc[preds['Prediction'] == score]
    preds = preds[~preds.URL.str.contains('|'.join(discard))]
    preds = preds[~preds.Title.str.contains('|'.join(discard))]
    preds = preds.sort_values(by='Score', ascending=False).reset_index(drop=True)
    preds.to_csv(f'{path}LOGREG_RELEVANCE/PREDICTIONS/{savefile}.csv')
    print(preds)
    return preds

def tweets_to_classify(path, filetype):   
    """ Merge all tweet searches together.
    
    Parameters
    ----------
    path: 
        for raw searches folder
    filetype: 
        the ending of the files to load, you can call just .pkl or also the date tag from file names
    """  
    raw_searches = path+'TWITTER_SEARCHES/RAW_SEARCHES/'
    result = pd.DataFrame()
    tweets_to_classify = pd.DataFrame()
    for file in os.listdir(raw_searches):
        if file.endswith(filetype):
            result = pd.read_pickle(raw_searches+file)
        tweets_to_classify = pd.concat([tweets_to_classify, result])
        tweets_to_classify = tweets_to_classify.reset_index(drop=True)
    print('Total tweets to classify:', len(tweets_to_classify))
    return tweets_to_classify

## Training twitter and descriptions classifiers

This is a ONE TIME operation. The models are pickled and loaded later to predict new results

In [ ]:
# one time training on twitter
twitter_training_model = LogRegSearches.train(twitter_training_set_v1, 'tweet', 'Target', 150, 'precision', 10, 'twitter_v1_150maxfeats', path)

# one time training on resources
#resource_training_model = lr_training(archive_desc_training_v2, 'Description', 'Target', 10, 'f1','resources_v2',path)

## Query Twitter

Calls Twitter API with the list of keywords and returns the table `prediction_twitter`

In [ ]:
#load token
token = 'AAAAAAAAAAAAAAAAAAAAAJgsNAEAAAAAQcsgbUnOJJmqmU483%2F8x6n9V1i8%3Df0qaEo9cV1sWP4eyNQ6E9s8BiRjvFTSN9mSqithe8uIXSNP68x'
#a selection of keywords based on MJI and musoW datasets

#choose KW list 
##OG keyword list
keywords = ['oral history', 'music magazine', 'sound archive', 'music history', 'music culture', 'music research', 'sheet music', 'music library', 'digital library', 'music collection', 'digital collection', 'sound recording', 'midi file', 'audio file', 'music information', 'musical score', 'digital score', 'song dataset', 'digital edition', 'digital archive', 'digital library', 'music archive', 'music library', 'archive collection']

##best performing list based on v1 tests
better_keywords = ['audio file', 'music archive', 'music collection', 'music library', 'sheet music', 'sound archive', 'sound recording']

#custom timeframe for searching
start = ['2022-01-01T00:00:00.000Z', '2022-02-01T00:00:00.000Z', '2022-03-01T00:00:00.000Z']
end = ['2022-01-31T23:59:59.000Z', '2022-02-28T23:59:59.000Z', '2022-03-31T23:59:59.000Z']

#choose search option 
## search last week
#twitter_search_weekly(token, better_keywords, 500, 500)
## search custom timeframe
twitter_search_custom(token, better_keywords, start, end, 500, 500)

In [ ]:
#load token
token = 'AAAAAAAAAAAAAAAAAAAAAJgsNAEAAAAAQcsgbUnOJJmqmU483%2F8x6n9V1i8%3Df0qaEo9cV1sWP4eyNQ6E9s8BiRjvFTSN9mSqithe8uIXSNP68x'
#a selection of keywords based on MJI and musoW datasets

#OG keyword list
keywords = ['oral history', 'music magazine', 'sound archive', 'music history', 'music culture', 'music research', 'sheet music', 'music library', 'digital library', 'music collection', 'digital collection', 'sound recording', 'midi file', 'audio file', 'music information', 'musical score', 'digital score', 'song dataset', 'digital edition', 'digital archive', 'digital library', 'music archive', 'music library', 'archive collection']

#best performing list based on v1 tests
better_keywords = ['audio file', 'music archive', 'music collection', 'music library', 'sheet music', 'sound archive', 'sound recording']

#transfer keywords to input list w/ additional twitter parameters, e.g. no retweets
input_keywords = [f'\"{k}\" -is:retweet' for k in keywords] 
#input time periods for search as a comma separated list, searches will be conducted for each time pair e.g. first of week/month, last of week/month 
start = ['2022-05-01T00:00:00.000Z', '2022-05-02T00:00:00.000Z', '2022-05-03T00:00:00.000Z', '2022-05-04T00:00:00.000Z', '2022-05-05T00:00:00.000Z', '2022-05-06T00:00:00.000Z', '2022-05-07T00:00:00.000Z']
end = ['2022-05-01T23:59:59.000Z', '2022-05-02T23:59:59.000Z', '2022-05-03T23:59:59.000Z', '2022-05-04T23:59:59.000Z', '2022-05-05T23:59:59.000Z', '2022-05-06T23:59:59.000Z', '2022-05-07T23:59:59.000Z']


#input max results / counts, and path 
mresults = 500 # max tweets per json response (100-500)
mcount = 500 # max tweets per search period 
#run the search! 
for k in input_keywords:
    filename = re.sub(r"([^A-Za-z0-9]+)", '', k) + f'_{start[0][0:10]}' + f'_{end[-1][6:10]}'
    filename = re.sub(r"isretweet", '', filename)
    prediction_twitter = LogRegSearches.search_twitter(token, k, start, end, mresults, mcount, filename)

#today = date.today()
#week_ago = today - timedelta(days=7)
#start = [week_ago.strftime("%Y-%m-%dT%H:%M:%S.000Z")]
#end = [today.strftime("%Y-%m-%dT%H:%M:%S.000Z")]

## Classify tweets

In [22]:
#load all search results into a single dataframe 
classified_tweets = LogRegSearches.tweets_to_classify(path, '2022-01-01_3-31.pkl')

Total tweets to classify: 2466


In [28]:
#run classification and get links from results
predicted_tweets, twitter_link_list = LogRegSearches.predict_twitter(path, 'twitter_v1_10maxfeats', classified_tweets, 'tweet', 1, '')

Total tweets classified: 611


In [10]:
predicted_tweets

,tweet,Prediction,Score,Probability,Input Length,URL,Search KW,tweet date
0,https://t.co/OkEHI6XQu3 Music archive Wolfgang...,1,14.077947,0.999999,139,http://www.opusip.co.uk/2022/01/25/music-archi...,"""music archive"" -is:retweet",2022-01-26
1,IUMA (Internet Underground Music Archive) Coll...,1,13.847835,0.999999,76,https://ift.tt/33IiRug,"""music archive"" -is:retweet",2022-01-24
2,IUMA (Internet Underground Music Archive) Coll...,1,13.847835,0.999999,80,https://ift.tt/3Asqwc9,"""music archive"" -is:retweet",2022-01-24
3,Music archive Wolfgang's Vault resolves copyri...,1,13.728025,0.999999,113,https://www.reuters.com/legal/transactional/mu...,"""music archive"" -is:retweet",2022-01-25
4,Music archive Wolfgang's Vault resolves copyri...,1,13.728025,0.999999,103,https://www.reuters.com/legal/transactional/mu...,"""music archive"" -is:retweet",2022-01-25
...,...,...,...,...,...,...,...,...
603,Wreckers of Civilization : Marcus Werner Hed a...,1,0.039432,0.509857,281,http://dlvr.it/SKmCvD,"""oral history"" -is:retweet",2022-02-27
604,"is reading ""https://t.co/s35k9d1vtn"" https://...",1,0.034007,0.508501,61,"https://ift.tt/GneE8yI, https://ift.tt/jaWQXrt","""oral history"" -is:retweet",2022-02-27
605,Our special ProsweetsCologne edition of Intern...,1,0.028407,0.507101,242,https://in-confectionery.com/january-2022-sing...,"""digital edition"" -is:retweet",2022-01-30
606,How annoying Kim. Check out https://t.co/pHXp...,1,0.018098,0.504524,119,https://www.bt.com/help/tv/fix-a-problem/recor...,"""sound recording"" -is:retweet",2022-03-29


## Scrape URLS

In [11]:
#scrape URL list and return a DF for resource classification
scraped_links = LogRegSearches.scrape_links(twitter_link_list, predicted_tweets, 'all_keywords_2022-01-01_03-31_scrapes_maxfeats100')

1 http://www.opusip.co.uk/2022/01/25/music-archive-wolfgangs-vault-resolves-copyright-fight-over-concert-recordings-2022-01-25/


Your max_length is set to 120, but you input_length is only 99. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=49)


2 https://ift.tt/33IiRug


Your max_length is set to 120, but you input_length is only 99. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=49)


3 https://ift.tt/3Asqwc9
4 https://www.reuters.com/legal/transactional/music-archive-wolfgangs-vault-resolves-copyright-fight-over-concert-recordings-2022-01-25/?utm_source=dlvr.it&utm_medium=twitter
5 https://www.reuters.com/legal/transactional/music-archive-wolfgangs-vault-resolves-copyright-fight-over-concert-recordings-2022-01-25/
6 https://archive.org/details/iuma-archive
7 https://pointblankdigital.shop/british-library-sound-archive-to-archive-our-music-catalogue/


...

412 http://dlvr.it/SKm8lW
413 https://bit.ly/TennisScrapbook65
414 https://okt.to/TQ3ebH
415 https://buff.ly/3tsZccs
416 https://www.bmwcca.org/content/oral-history-project-announcement
417 https://mailchi.mp/f03ec20b9f7c/pre-pesach


Your max_length is set to 120, but you input_length is only 111. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=55)


418 http://dlvr.it/SKmCvD


Your max_length is set to 120, but you input_length is only 23. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)


419 https://www.bt.com/help/tv/fix-a-problem/recording-or-sound/recording-problems-on-bt-tv#:~:text=Here%20are%20some%20steps%20to%20help%20stop%20this%20from%20happening%20again%3A&text=Check%20your%20equipment%20cabling%20is,%27%20section%20of%20%27My%20TV%27
386


In [16]:
scraped_links = pd.read_pickle(path+'LOGREG_RELEVANCE/SCRAPES/all_keywords_2022-01-01_03-31_scrapes_v1.pkl')

In [17]:
len(scraped_links) 

418

## Classify URLS

In [18]:
predicted_resources = LogRegSearches.predict_resource(path, 'resources_v2', scraped_links, 'Description', 1, 'all_keywords_2022-01-01_03-31_v1')

In [19]:
predicted_resources

,tweet,Prediction,Score,Probability,Input Length,URL,Search KW,tweet date,Title,Description
0,Jamsphere Indie Music Magazine January 2022 - ...,1,4.181303,0.984951,195,https://jamsphere.com/printed-magazine-issues/...,"""music magazine"" -is:retweet",2022-01-29,Jamsphere Indie Music Magazine January 2022 | ...,Jamsphere Indie Music Magazine January 2022 –...
1,Project: Music Recommendation Million Song ...,1,3.689997,0.975636,319,http://millionsongdataset.com,"""song dataset"" -is:retweet",2022-03-29,Welcome! | Million Song Dataset,The Million Song Dataset is a free-available ...
2,2017 @ the British Library Sound Archive. Grat...,1,3.573228,0.972701,261,https://www.teresabarrozo.com/post/the-british...,"""sound archive"" -is:retweet",2022-01-21,@ the British Library Sound Achive,The Wild Life and Environmental Sounds Archiv...
3,Jamsphere Indie Music Magazine March 2022 - ht...,1,3.344960,0.965939,182,https://jamsphere.com/printed-magazine-issues/...,"""music magazine"" -is:retweet",2022-03-30,Jamsphere Indie Music Magazine March 2022 | Ja...,Jamsphere Indie Music Magazine March 2022 – F...
4,Jamsphere Indie Music Magazine February 2022 -...,1,3.204508,0.961004,184,https://jamsphere.com/printed-magazine-issues/...,"""music magazine"" -is:retweet",2022-02-26,Jamsphere Indie Music Magazine February 2022 |...,Jamsphere Indie Music Magazine February 2022 ...
...,...,...,...,...,...,...,...,...,...,...
164,Fire Island time machine: This recently discov...,1,0.080876,0.520208,291,https://bit.ly/3oPyZBt,"""music archive"" -is:retweet",2022-02-16,Fagulous Fire Island Music Archive + Eric Adam...,A huge archive of tapes discovered in a recen...
165,The Legend of Vox Machina Musical Score | Prim...,1,0.075942,0.518976,333,https://binge.place/amazon-prime/the-legend-of...,"""musical score"" -is:retweet",2022-03-28,Binge Place,The Legend of Vox Machina is more interested ...
166,The Legend of Vox Machina Musical Score | Prim...,1,0.075942,0.518976,353,http://dlvr.it/SMWwSw,"""musical score"" -is:retweet",2022-03-28,The Legend of Vox Machina Musical Score | Prim...,The Legend of Vox Machina is more interested ...
167,Turning on the tunes: 3 evidence-based benefit...,1,0.008282,0.502071,606,https://bit.ly/3rrgHZ5,"""music research"" -is:retweet",2022-02-25,Turning on the tunes: 3 evidence-based benefit...,"Music therapy, a safe and non-invasive comple..."
